In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt 
import tensorflow as tf

import textstat
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import torch
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
#from transformers import AdamW
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import load_model
import json


C:\Users\mooli\AppData\Local\Temp\ipykernel_18084\2529265094.py:30: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import AdamW
#from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [3]:
df_tr_01 = pd.read_csv('training_data.csv')
print(len(df_tr_01))
print(df_tr_01.head())

4800
   id                                           sentence difficulty
0   0  Les coûts kilométriques réels peuvent diverger...         C1
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2   2  Le test de niveau en français est sur le site ...         A1
3   3           Est-ce que ton mari est aussi de Boston?         A1
4   4  Dans les écoles de commerce, dans les couloirs...         B1


In [4]:
df_unte_01 = pd.read_csv('unlabelled_test_data.csv')
print(len(df_unte_01))
print(df_unte_01.head())

1200
   id                                           sentence
0   0  Nous dûmes nous excuser des propos que nous eû...
1   1  Vous ne pouvez pas savoir le plaisir que j'ai ...
2   2  Et, paradoxalement, boire froid n'est pas la b...
3   3  Ce n'est pas étonnant, car c'est une saison my...
4   4  Le corps de Golo lui-même, d'une essence aussi...


In [5]:
print(df_tr_01.shape)
print(df_tr_01.info())
print('----------')
df_tr_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_02.iloc[i,2] == "A1":
        df_tr_02.iloc[i,2] = int(1)
    elif df_tr_02.iloc[i,2] == "A2":
        df_tr_02.iloc[i,2] =int(2)
    elif df_tr_02.iloc[i,2] == "B1":
        df_tr_02.iloc[i,2] = int(3)
    elif df_tr_02.iloc[i,2] == "B2":
        df_tr_02.iloc[i,2] = int(4)
    elif df_tr_02.iloc[i,2] == "C1":
        df_tr_02.iloc[i,2] = int(5)
    elif df_tr_02.iloc[i,2] == "C2":
        df_tr_02.iloc[i,2] = int(6)
df_tr_02.difficulty.unique()

df_tr_05_02 = df_tr_01.copy()
for i in range(0, len(df_tr_02.difficulty)):
    if df_tr_05_02.iloc[i,2] == "A1":
        df_tr_05_02.iloc[i,2] = int(0)
    elif df_tr_05_02.iloc[i,2] == "A2":
        df_tr_05_02.iloc[i,2] =int(1)
    elif df_tr_05_02.iloc[i,2] == "B1":
        df_tr_05_02.iloc[i,2] = int(2)
    elif df_tr_05_02.iloc[i,2] == "B2":
        df_tr_05_02.iloc[i,2] = int(3)
    elif df_tr_05_02.iloc[i,2] == "C1":
        df_tr_05_02.iloc[i,2] = int(4)
    elif df_tr_05_02.iloc[i,2] == "C2":
        df_tr_05_02.iloc[i,2] = int(5)
df_tr_05_02.difficulty.unique()


(4800, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4800 non-null   int64 
 1   sentence    4800 non-null   object
 2   difficulty  4800 non-null   object
dtypes: int64(1), object(2)
memory usage: 112.6+ KB
None
----------


array([4, 0, 2, 1, 3, 5], dtype=object)

## stemmer

In [ ]:
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Initialize French stemmer
stemmer = SnowballStemmer("french")

# French stopwords from NLTK
french_stopwords = set(stopwords.words('french'))

def clean_text_french(text):
    # Clean and prepare text
    text = re.sub("[^\w\s]", " ", text)
    # Split text into words, filter stopwords, apply stemming, and convert to lower case
    return " ".join([stemmer.stem(word) for word in text.split() if word.lower() not in french_stopwords]).lower()

'''
# Sample data
data = pd.DataFrame({
    'text': ["Café au centre de Paris", "Il fait un beau jour!", "résumé et portfolio"]
})
'''
df_tr_03 = df_tr_02.copy()

# Applying the French text cleaning function
df_tr_03['cleaned'] = df_tr_03['sentence'].apply(clean_text_french)
print(df_tr_03.cleaned)

## lemmatize

In [ ]:
import spacy

# Load French Spacy model
nlp = spacy.load("fr_core_news_sm")

def clean_text_french_lemmatized(text):
    # Process text using Spacy
    doc = nlp(text)
    # Filter stopwords and punctuation, then lemmatize
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]).lower()

df_tr_04 = df_tr_02.copy()

# Applying the French text cleaning function with lemmatization
df_tr_04['cleaned'] = df_tr_04['sentence'].apply(clean_text_french_lemmatized)
print(df_tr_04.cleaned)


 ## ML

### Camembert embeddings

In [8]:
#camembert embeddings

import pandas as pd
import torch
from transformers import CamembertTokenizer, CamembertModel
from sklearn.preprocessing import StandardScaler

df_tr_20 = df_tr_02.copy()

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

def get_embeddings(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    encoded_input = {key: val.to(model.device) for key, val in encoded_input.items()}
    with torch.no_grad():
        outputs = model(**encoded_input)
    #hidden state
    embeddings = outputs.last_hidden_state
    embeddings = embeddings.mean(dim=1)
    return embeddings.squeeze().cpu().tolist()

df_tr_20['embeddings'] = df_tr_20['sentence'].apply(get_embeddings)
#print(df_tr_20['embeddings'].shape())
df_tr_20.head()


f:\projects\Matthias\proj_01\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,id,sentence,difficulty,embeddings
0,0,Les coûts kilométriques réels peuvent diverger...,5,"[0.04393050819635391, -0.012339390814304352, 0..."
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",1,"[0.03156255558133125, -0.06494396924972534, -0..."
2,2,Le test de niveau en français est sur le site ...,1,"[0.01727151870727539, -0.05883779004216194, -0..."
3,3,Est-ce que ton mari est aussi de Boston?,1,"[0.0041115740314126015, -0.07472144812345505, ..."
4,4,"Dans les écoles de commerce, dans les couloirs...",3,"[0.025271179154515266, 0.0095027731731534, -0...."


In [9]:
print(df_tr_20['embeddings'].shape)

(4800,)


### Multi input NN

In [12]:
import json
#si weight and biases account
'''
import wandb
run = wandb.init(
    project='Cam02', 
    name="001", 
)
'''

df_tr_08 = df_tr_01.copy()
label_encoder = LabelEncoder()
df_tr_08['difficulty'] = label_encoder.fit_transform(df_tr_08['difficulty'])

nlp = spacy.load('fr_core_news_sm')

def extract_features(doc):
    # Lexical features
    words = [token.text for token in doc if token.is_alpha]
    avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
    type_token_ratio = len(set(words)) / len(words) if words else 0
    
    # Lexical density
    content_pos_tags = {'NOUN', 'VERB', 'ADJ', 'ADV'}
    content_words = [token for token in doc if token.pos_ in content_pos_tags]
    lexical_density = len(content_words) / len(words) if words else 0
    
    # Syntactic features
    sentences = list(doc.sents)
    avg_sentence_length = sum(len(sentence) for sentence in sentences) / len(sentences) if sentences else 0
    
    #Syntactic complexity: estimated by counting finite verbs in each sentence
    clauses_per_sentence = [sum(1 for token in sentence if token.tag_ in ['VERB', 'AUX']) for sentence in sentences]
    avg_clauses_per_sentence = sum(clauses_per_sentence) / len(clauses_per_sentence) if clauses_per_sentence else 0
    
    noun_phrases = list(doc.noun_chunks)
    avg_noun_phrase_length = sum(len(np) for np in noun_phrases) / len(noun_phrases) if noun_phrases else 0
    
    #Count of nouns, verbs, adjectives
    num_nouns = sum(1 for token in doc if token.pos_ == 'NOUN')
    num_verbs = sum(1 for token in doc if token.pos_ == 'VERB')
    num_adjectives = sum(1 for token in doc if token.pos_ == 'ADJ')
    
    return {
        'avg_word_length': avg_word_length,
        'type_token_ratio': type_token_ratio,
        'lexical_density': lexical_density,
        'avg_sentence_length': avg_sentence_length,
        'avg_clauses_per_sentence': avg_clauses_per_sentence,
        'avg_noun_phrase_length': avg_noun_phrase_length,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjectives': num_adjectives
    }

def add_features_to_dataframe(df, text_column):
    df['spacy_doc'] = df[text_column].apply(nlp)  # Convert text to spacy Doc objects
    features_df = df['spacy_doc'].apply(extract_features).apply(pd.Series)
    new_df = pd.concat([df, features_df], axis=1)
    new_df.drop('spacy_doc', axis=1, inplace=True)
    return new_df

df_with_features_01 = add_features_to_dataframe(df_tr_08, 'sentence')

df_with_features_02 = df_with_features_01.copy()
df_with_features_02['embeddings'] = df_tr_20['embeddings'] #df_tr_20 les embed camembert
df_with_features_02 = df_with_features_02.drop('sentence', axis=1)
df_with_features_02 = df_with_features_02.drop('id', axis=1)
df_only_features_02 = df_with_features_02.drop('difficulty', axis=1)

df_y_01 = df_tr_05_02.copy()
df_y_01['difficulty'] = df_y_01['difficulty'].astype(float)
print('----------------------------------------------')

X_train, X_test, y_train, y_test = train_test_split(df_only_features_02, df_y_01['difficulty'], test_size=0.1, random_state=42)

X_train_embeddings = X_train['embeddings'].tolist()
X_train_features = X_train.drop('embeddings', axis=1)

X_test_embeddings = X_test['embeddings'].tolist()
X_test_features = X_test.drop('embeddings', axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_test_scaled = scaler.fit_transform(X_test_features) ######################## transform

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_features.columns, index=X_train_features.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_features.columns, index=X_test_features.index)
#---------------------------------------------------------
#----------------------------------------------------------------
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train_scaled, y_train)

importances = forest.feature_importances_
important_feature_indices = np.argsort(importances)[::-1][:4]  # Select top 3 features for simplicity

important_features = X_train_scaled.columns[important_feature_indices]

X_train_numerical_selected = X_train_scaled[important_features]
X_test_numerical_selected = X_test_scaled[important_features]

'''
pca = PCA(n_components=800)  # Reduce to 50 components
X_train_embeddings_reduced = pca.fit_transform(X_train_embeddings)
X_test_embeddings_reduced = pca.transform(X_test_embeddings)
'''
#----------------------------------------------------------------
'''
numeric_input = Input(shape=(X_train_numerical_selected.shape[1],), name='numeric_data')
embedding_input = Input(shape=(len(X_train_embeddings[0]),), name='embedding_data')
'''

print("Shape of X_train_numerical_selected:", X_train_numerical_selected.shape)

# Input layers
input_numerical = Input(shape=(X_train_numerical_selected.shape[1],), name='numeric_input')
#input_embeddings = Input(shape=(800,), name='embedding_input')
input_embeddings = Input(shape=(np.array(X_train_embeddings).shape[1],), name='embedding_input')

# Processing paths
path_numerical = Dense(50, activation='relu')(input_numerical)
path_embeddings = Dense(50, activation='relu')(input_embeddings)

# Concatenate data paths
concatenated = Concatenate()([path_numerical, path_embeddings])

# Additional Dense layers
dense = Dense(50, activation='relu')(concatenated)
dense = Dense(25, activation='relu')(dense) 
#dropout = Dropout(0.5)(dense)
output = Dense(6, activation='softmax')(dense)

optimizer = AdamW(learning_rate=0.0001) # 0.01

model = Model(inputs=[input_numerical, input_embeddings], outputs=output)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

X_train_embeddings_reduced_np = np.array(X_train_embeddings)
X_test_embeddings_reduced_np = np.array(X_test_embeddings)

#print("Shape of X_train_numerical_selected:", X_train_numerical_selected.shape)
#print("Shape of X_train_embeddings_reduced:", X_train_embeddings_reduced.shape)

model.fit([X_train_numerical_selected, X_train_embeddings_reduced_np], y_train, epochs=8, validation_split=0.1, batch_size=1) #50 0.1 4

----------------------------------------------
Shape of X_train_numerical_selected: (4320, 4)
Epoch 1/8
3888/3888 ━━━━━━━━━━━━━━━━━━━━ 1:01:45 953ms/step - accuracy: 0.0000e+00 - loss: 1.853 ━━━━━━━━━━━━━━━━━━━━ 3s 912us/step - accuracy: 0.1238 - loss: 1.7953        ━━━━━━━━━━━━━━━━━━━━ 3s 856us/step - accuracy: 0.1756 - loss: 1.79 ━━━━━━━━━━━━━━━━━━━━ 3s 828us/step - accuracy: 0.1932 - loss: 1.79 ━━━━━━━━━━━━━━━━━━━━ 2s 816us/step - accuracy: 0.2023 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 2s 810us/step - accuracy: 0.2079 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 2s 809us/step - accuracy: 0.2161 - loss: 1.77 ━━━━━━━━━━━━━━━━━━━━ 2s 809us/step - accuracy: 0.2238 - loss: 1.77 ━━━━━━━━━━━━━━━━━━━━ 2s 804us/step - accuracy: 0.2326 - loss: 1.76 ━━━━━━━━━━━━━━━━━━━━ 2s 803us/step - accuracy: 0.2404 - loss: 1.76 ━━━━━━━━━━━━━━━━━━━━ 2s 822us/step - accuracy: 0.2456 - loss: 1.76 ━━━━━━━━━━━━━━━━━━━━ 2s 829us/step - accuracy: 0.2504 - loss: 1.75 ━━━━━━━━━━━━━━━━━━━━ 2s 834us/step - accuracy: 0.2546 - loss: 1

"pos_tags = {token.pos: token.pos_ for token in doc}\n\nreadability_score = textstatistics().flesch_reading_ease(text).drop('spacy_doc', axis=1, inplace=True)  # Optionally drop the Spacy Doc column\n    return new_df\n\n\n# Adding features to DataFrame\ndf_with_features_01 = add_features_to_dataframe(df_tr_08, 'sentence')\n\ndf_with_fe\n"

In [13]:
test_loss, test_accuracy = model.evaluate([(X_test_numerical_selected, X_test_embeddings_reduced_np)], y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.5625 - loss: 0.887 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5723 - loss: 0.9888 
Test Loss: 0.9919983744621277
Test Accuracy: 0.5791666507720947


In [ ]:
model.save('my_model.h5')

### prepare unabelled data

In [ ]:
#save unte

df_tr_un_08 = df_unte_01.copy()

label_encoder = LabelEncoder()

nlp = spacy.load('fr_core_news_sm')

#WIP use only the selected features
'''
def extract_features(doc):
    # Lexical features
    words = [token.text for token in doc if token.is_alpha]
    avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
    type_token_ratio = len(set(words)) / len(words) if words else 0
    
    # Lexical density: proportion of content words (nouns, verbs, adjectives, adverbs)
    content_pos_tags = {'NOUN', 'VERB', 'ADJ', 'ADV'}
    content_words = [token for token in doc if token.pos_ in content_pos_tags]
    lexical_density = len(content_words) / len(words) if words else 0
    
    # Syntactic features
    sentences = list(doc.sents)
    avg_sentence_length = sum(len(sentence) for sentence in sentences) / len(sentences) if sentences else 0
    
    # Syntactic complexity: estimated by counting finite verbs in each sentence
    clauses_per_sentence = [sum(1 for token in sentence if token.tag_ in ['VERB', 'AUX']) for sentence in sentences]
    avg_clauses_per_sentence = sum(clauses_per_sentence) / len(clauses_per_sentence) if clauses_per_sentence else 0
    
    noun_phrases = list(doc.noun_chunks)
    avg_noun_phrase_length = sum(len(np) for np in noun_phrases) / len(noun_phrases) if noun_phrases else 0
    
    # Semantic features: Count of nouns, verbs, adjectives
    num_nouns = sum(1 for token in doc if token.pos_ == 'NOUN')
    num_verbs = sum(1 for token in doc if token.pos_ == 'VERB')
    num_adjectives = sum(1 for token in doc if token.pos_ == 'ADJ')
    
    # Return all features as a dictionary
    return {
        'avg_word_length': avg_word_length,
        'type_token_ratio': type_token_ratio,
        'lexical_density': lexical_density,
        'avg_sentence_length': avg_sentence_length,
        'avg_clauses_per_sentence': avg_clauses_per_sentence,
        'avg_noun_phrase_length': avg_noun_phrase_length,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjectives': num_adjectives
    }
'''
def add_features_to_dataframe(df, text_column):
    df['spacy_doc'] = df[text_column].apply(nlp)
    features_df = df['spacy_doc'].apply(extract_features).apply(pd.Series)
    new_df = pd.concat([df, features_df], axis=1)
    new_df.drop('spacy_doc', axis=1, inplace=True)
    return new_df

df_un_with_features_01 = add_features_to_dataframe(df_tr_un_08, 'sentence')
df_un_with_features_02 = df_un_with_features_01.copy()
df_un_with_features_02['embeddings'] = df_tr_20['embeddings']
df_un_with_features_02 = df_un_with_features_02.drop('sentence', axis=1)
df_un_with_features_02 = df_un_with_features_02.drop('id', axis=1)

X_train_embeddings = df_un_with_features_02['embeddings'].tolist()
X_train_features = df_un_with_features_02.drop('embeddings', axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_features.columns, index=X_train_features.index)
'''
# Get feature importance and select the most important ones
importances = forest.feature_importances_
important_feature_indices = np.argsort(importances)[::-1][:3]  # Select top N features

important_features = X_train_scaled.columns[important_feature_indices]

# Filter your train and test sets
X_train_numerical_selected = X_train_scaled[:, important_feature_indices]
X_test_numerical_selected = X_test_scaled[:, important_feature_indices]
'''
# Get feature importance and sort them
#importances = forest.feature_importances_
#important_feature_indices = np.argsort(importances)[::-1][:4]  # Select top 3 features for simplicity

important_features = X_train_scaled.columns[important_feature_indices]

X_train_numerical_selected = X_train_scaled[important_features]

In [ ]:
loaded_model_h5 = tf.keras.models.load_model('my_model.h5')
predictions = loaded_model_h5.predict([X_new_numerical, X_new_embeddings])

In [ ]:
# Predict using the trainer
predicted_labels = np.argmax(predictions.predictions, axis=1)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_labels]

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predicted_difficulties
})

submission_df.to_csv('F:\projects\Matthias\proj_01\cam01\submission_CamemBERT_018.csv', index=False)

In [14]:
print("Selected features:", important_features)

Selected features: Index(['avg_word_length', 'avg_sentence_length', 'lexical_density',
       'avg_noun_phrase_length'],
      dtype='object')


In [35]:
print(X_train_numerical_selected.shape, np.array(X_train_embeddings).shape, y_train.shape)

(4320, 4) (4320, 768) (4320,)


In [37]:
print(X_test_numerical_selected.shape, np.array(X_test_embeddings).shape, y_test.shape)

(480, 4) (480, 768) (480,)


### Hypermodel

In [15]:

'''
def apply_pca(X, n_components=666): #50
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca, pca
'''

class FullHyperModel(HyperModel):
    def __init__(self, X_train_num, X_train_emb, y_train):
        self.X_train_num = X_train_num
        self.X_train_emb = X_train_emb
        self.y_train = y_train

    def build(self, hp):
        # PCA for embeddings
        #n_components = hp.Int('n_components', min_value=100, max_value=1000, step=20)
        
        '''
        pca = PCA(n_components=n_components)
        X_train_emb_pca = pca.fit_transform(self.X_train_emb)
        '''

    
        input_emb = Input(shape=(self.X_train_emb.shape[1],), name='input_emb')
        emb_units = hp.Int('emb_units', min_value=20, max_value=256, step=32)
        emb_path = Dense(emb_units, activation='relu')(input_emb)

        # Input for numerical features
        input_num = Input(shape=(self.X_train_num.shape[1],), name='input_num')
        num_units = hp.Int('num_units', min_value=20, max_value=256, step=32)
        num_path = Dense(num_units, activation='relu')(input_num)

        # Concatenate and further processing
        concatenated = Concatenate()([num_path, emb_path])
        final_units = hp.Int('final_units', min_value=10, max_value=256, step=32)
        dense = Dense(final_units, activation='relu')(concatenated)
        
        output = Dense(6, activation='softmax')(dense)  
        
        # Hyperparameters for the optimizer
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        weight_decay = hp.Float('weight_decay', min_value=1e-5, max_value=1e-2, step=1e-5)

        # Model compilation
        model = Model(inputs=[input_num, input_emb], outputs=output)
        '''
        model.compile(
            optimizer=AdamW(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]),weight_decay=hp.Float('weight_decay', min_value=0.01, max_value=0.1, step=0.01)),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        '''
        model.compile(
            optimizer=tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
            # optimizer=AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        model.summary()
        return model
    

    def fit(self, hp, model, *args, **kwargs):
        batch_size = hp.Int('batch_size', min_value=1, max_value=16)
        #n_components = hp.get('n_components')
        #pca = PCA(n_components=n_components)
        #X_train_emb_pca = pca.fit_transform(self.X_train_emb)
        print("Training data shape:", self.X_train_num.shape)
        print("Training embeddings shape:", self.X_train_emb.shape)
        print("Labels shape:", self.y_train.shape)
        print("batch size:", batch_size)
        return model.fit([self.X_train_num, self.X_train_emb], self.y_train, batch_size=batch_size, *args, **kwargs)
    
tuner = RandomSearch(
    FullHyperModel(X_train_numerical_selected, np.array(X_train_embeddings), y_train),
    objective='val_accuracy',
    max_trials=100, 
    executions_per_trial=4, 
    directory='my_tuning_dir',
    project_name='full_model_tuning_11'
)

tuner.search(epochs=30, validation_split=0.1)

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Trial 23 Complete [00h 02m 42s]
val_accuracy: 0.5353009253740311

Best val_accuracy So Far: 0.5486111342906952
Total elapsed time: 01h 22m 11s

Search: Running Trial #24

Value             |Best Value So Far |Hyperparameter
212               |84                |emb_units
180               |180               |num_units
202               |74                |final_units
0.0030721         |0.00036557        |learning_rate
0.00866           |0.00883           |weight_decay
1                 |14                |batch_size



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_num           │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_emb           │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 180)       │        900 │ input_num[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 212)       │    163,028 │ input_emb[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 392)       │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 202)       │     79,386 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 6)         │      1,218 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 244,532 (955.20 KB)

 Trainable params: 244,532 (955.20 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_num           │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_emb           │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 180)       │        900 │ input_num[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 212)       │    163,028 │ input_emb[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 392)       │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 202)       │     79,386 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 6)         │      1,218 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 244,532 (955.20 KB)

 Trainable params: 244,532 (955.20 KB)

 Non-trainable params: 0 (0.00 B)

Training data shape: (4320, 4)
Training embeddings shape: (4320, 768)
Labels shape: (4320,)
batch size: 1
Epoch 1/30
3888/3888 ━━━━━━━━━━━━━━━━━━━━ 53:42 829ms/step - accuracy: 0.0000e+00 - loss: 1.941 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.0477 - loss: 2.0168        ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1043 - loss: 1.95 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1379 - loss: 1.88 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1589 - loss: 1.84 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1743 - loss: 1.81 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1858 - loss: 1.79 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.1965 - loss: 1.77 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2056 - loss: 1.76 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2140 - loss: 1.74 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2210 - loss: 1.72 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2279 - loss: 1.71 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.2339 - loss: 1.70 ━━━━━━━━━

In [149]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('best_model_03.h5')

In [150]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

batch_size = best_hp.get('batch_size')
learning_rate = best_hp.get('learning_rate')
weight_decay = best_hp.get('weight_decay')


# Optionally, directly fetch the best model using the best hyperparameters
#best_model = tuner.get_best_models(num_models=1)[0]

hyperparameters = {
    'batch_size': best_hp.get('batch_size'),
    'learning_rate': best_hp.get('learning_rate'),
    'weight_decay': best_hp.get('weight_decay'),
    #'n_components': best_hp.get('n_components')
}

with open('best_hyperparameters.json', 'w') as f:
    json.dump(hyperparameters, f)

In [151]:
print(X_train_numerical_selected.shape)
print(len(X_train_embeddings))

(4320, 4)
4320


In [152]:
model = load_model('best_model_03.h5')
# Load the hyperparameters
with open('best_hyperparameters.json', 'r') as f:
    hyperparams = json.load(f)

print(hyperparams['batch_size'], hyperparams['learning_rate'], hyperparams['weight_decay'])

#pca = PCA(n_components=700)  # Reduce to 50 components
#X_test_embeddings_reduced = pca.fit_transform(X_test_embeddings)

11 0.0001242567309905867 0.00397


In [154]:
#model.predict(X_test_numerical_selected, X_test_embeddings, y_test)
test_loss, test_accuracy = model.evaluate([(X_test_numerical_selected, pd.DataFrame(X_test_embeddings))], y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5825 - loss: 1.0569  
Test Loss: 1.059268593788147
Test Accuracy: 0.5854166746139526


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

predictions = best_model.predict([X_test_numerical_selected, pd.DataFrame(X_test_embeddings)])
predicted_classes = np.argmax(predictions, axis=1)
difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_classes]
original_difficulties = [difficulty_levels[label] for label in y_test]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()